In [20]:
import os
os.environ["SQLITE_BUSY_TIMEOUT"] = "30000"  # 30 segundos

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, minmax_scale, scale
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader

In [2]:
data_path= "/home/21953404Victor/Deep/drive-download-20231121T165626Z-001/HotelReservationsPreparedCleanAttributes.csv"
label_path = "/home/21953404Victor/Deep/drive-download-20231121T165626Z-001/HotelReservationsOutput.csv"

In [3]:
features  = pd.read_csv(data_path)
labels = pd.read_csv(label_path)

In [4]:
from sklearn.model_selection import train_test_split
# Divide el conjunto de datos
features_train, features_test, labels_train, labels_test = train_test_split(features.values, labels.values, test_size=0.1, random_state=42)
features_train, features_val, labels_train, labels_val = train_test_split(features_train, labels_train, test_size=0.1, random_state=42)

# Convierte los arrays a tensores de PyTorch
features_train = torch.tensor(features_train).float()
labels_train = torch.tensor(labels_train).float()
features_val = torch.tensor(features_val).float()
labels_val = torch.tensor(labels_val).float()
features_test = torch.tensor(features_test).float()
labels_test = torch.tensor(labels_test).float()

# Crea conjuntos de datos de PyTorch
train_dataset = TensorDataset(features_train, labels_train)
val_dataset = TensorDataset(features_val, labels_val)
test_dataset = TensorDataset(features_test, labels_test)

# Crea DataLoaders de PyTorch
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
import tensorflow as tf
import torch

# Verificar si CUDA está disponible
cuda_available = torch.cuda.is_available()
print(f"CUDA disponible: {cuda_available}")

# Enumerar dispositivos de GPU disponibles y mostrar sus nombres si CUDA está disponible
if cuda_available:
    print("Listado de dispositivos GPU disponibles:")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No se detectó una GPU. Asegúrate de que el acelerador de hardware esté configurado.")


2023-11-30 15:15:12.758163: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-30 15:15:13.205005: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 15:15:14.286567: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/hadoop/lib/native:
2023-11-30 15:15:14.286641: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot

CUDA disponible: True
Listado de dispositivos GPU disponibles:
GPU 0: NVIDIA GeForce RTX 3080 Ti


In [6]:
import torch.nn as nn

class PyTorchModel(nn.Module):
    def __init__(self, trial, input_shape):
        super(PyTorchModel, self).__init__()
        self.layers = nn.ModuleList()
        self.l2_regs = []
        current_shape = input_shape

        # Definir la arquitectura de la red
        n_layers = trial.suggest_int('n_layers', 1, 50)
        for i in range(n_layers):
            num_units = trial.suggest_categorical(f'n_units_l{i}', [16, 32, 64, 128, 256, 512, 1024, 2048])
            activation = trial.suggest_categorical(f'activation_l{i}', ['ReLU', 'Tanh', 'ELU', 'SELU', 'LeakyReLU'])
            dropout_rate = trial.suggest_float(f'dropout_l{i}', 0.0, 0.5)
            use_batch_norm = trial.suggest_categorical(f'batch_norm_l{i}', [True, False])
            l2_reg = trial.suggest_float(f'l2_reg_l{i}', 1e-5, 1e-1, log=True)

            # Crear y añadir la capa lineal
            layer = nn.Linear(current_shape, num_units)
            self.layers.append(layer)
            self.l2_regs.append(l2_reg)  # Añadir la tasa de regularización L2

            current_shape = num_units

            # Añadir Batch Normalization si se indica
            if use_batch_norm:
                self.layers.append(nn.BatchNorm1d(num_units))
            
            # Añadir Dropout
            self.layers.append(nn.Dropout(dropout_rate))

            # Añadir la activación
            if activation == 'LeakyReLU':
                negative_slope = trial.suggest_float(f'leakyrelu_alpha_l{i}', 0.01, 0.3)
                self.layers.append(nn.LeakyReLU(negative_slope=negative_slope))
            else:
                self.layers.append(getattr(nn, activation)())

        # Capa de salida
        self.output = nn.Linear(current_shape, 1)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return torch.sigmoid(self.output(x))

    def get_l2_loss(self):
        l2_loss = 0.0
        for layer, l2_reg in zip(self.layers, self.l2_regs):
            if isinstance(layer, nn.Linear):
                l2_loss += l2_reg * layer.weight.norm(2)
        return l2_loss



In [7]:
from torch.optim import Adam, SGD, RMSprop
from torch.utils.data import DataLoader

def build_model(params, input_shape, is_trial=True):
    # Crear una instancia del modelo con la arquitectura definida
    model = PyTorchModel(params, input_shape)

    if is_trial:
        # Configuración durante la optimización con Optuna
        optimizer_name = params.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
        lr = params.suggest_float('lr', 1e-5, 1e-1, log=True)
        if optimizer_name == 'Adam':
            beta1 = params.suggest_float('adam_beta1', 0.85, 0.99)
            beta2 = params.suggest_float('adam_beta2', 0.9, 0.999)
            optimizer = Adam(model.parameters(), lr=lr, betas=(beta1, beta2))
        elif optimizer_name == 'SGD':
            momentum = params.suggest_float('sgd_momentum', 0.1, 0.9)
            optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
        elif optimizer_name == 'RMSprop':
            alpha = params.suggest_float('rmsprop_alpha', 0.85, 0.99)
            optimizer = RMSprop(model.parameters(), lr=lr, alpha=alpha)
    else:
        # Configuración con parámetros fijos
        optimizer_name = params['optimizer']
        if optimizer_name == 'Adam':
            optimizer = Adam(model.parameters(), lr=params['lr'], betas=(params['adam_beta1'], params['adam_beta2']))
        elif optimizer_name == 'SGD':
            optimizer = SGD(model.parameters(), lr=params['lr'], momentum=params.get('sgd_momentum', 0.9))
        elif optimizer_name == 'RMSprop':
            optimizer = RMSprop(model.parameters(), lr=params['lr'], alpha=params.get('rmsprop_alpha', 0.99))

    return model, optimizer



In [8]:
from tqdm import tqdm
import torch

def calculate_accuracy(outputs, labels):
    predicted = outputs.round()  # Redondear para obtener las predicciones binarias
    correct = (predicted == labels).sum().item()
    return correct / labels.size(0)

In [9]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import gc
from torch.utils.data import DataLoader

def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies, file_name_base):
    plt.figure(figsize=(12, 4))

    # Pérdida
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Pérdida durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()

    # Precisión
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.title('Precisión durante el entrenamiento')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()

    plt.savefig(f"{file_name_base}_plot.png")
    plt.close()

In [10]:
import json

best_accuracy_file = "global_best_accuracy.json"

# Intenta cargar la mejor precisión global desde un archivo
try:
    with open(best_accuracy_file, "r") as file:
        global_best_accuracy = json.load(file)
except (FileNotFoundError, json.JSONDecodeError, TypeError, ValueError):
    global_best_accuracy = 0.0
    
best_model_path = "best_model.pt"
plot_file_name_base = "best_model"

print(f"Mejor accuracy global: {global_best_accuracy}")

Mejor accuracy global: 0.8903522205206739


## Train

In [11]:
import torch
import gc
from torch.utils.data import DataLoader

def objective(trial):
    global global_best_accuracy, best_model_path, plot_file_name_base
    # Verificar y configurar el uso de la GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Usando {'GPU' if device.type == 'cuda' else 'CPU'} para el Trial {trial.number}")

    model, optimizer = build_model(trial, input_shape)  # Asegúrate de que 'input_shape' esté definido
    model.to(device)

    # Parametrización del número de épocas y tamaño del batch
    epochs = trial.suggest_int('epochs', 10, 2000)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024, 2048])

    # Criterio de pérdida y DataLoader
    criterion = torch.nn.BCELoss()  # Ajustar según tu problema
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Configuración de Early Stopping y Learning Rate Scheduler
    early_stopping_patience = trial.suggest_int('early_stopping_patience', 3, 20)
    factor = trial.suggest_float("lr_reduce_factor", 0.1, 0.9)
    lr_patience = trial.suggest_int("lr_reduce_patience", 5, 50)
    min_lr = trial.suggest_float("min_lr", 1e-6, 1e-4, log=True)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=factor, patience=lr_patience, min_lr=min_lr, verbose=True)

    # Entrenamiento del modelo
    print(f"Entrenando Trial {trial.number}...")
    best_accuracy = 0.0  # Inicializacion de la mejor precisión
    epochs_no_improve = 0
    # Inicializar listas para almacenar métricas
    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            train_correct += calculate_accuracy(outputs, labels) * inputs.size(0)
            train_total += labels.size(0)

        train_loss /= train_total
        train_accuracy = train_correct / train_total

        # Almacenar las métricas de entrenamiento
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        # Evaluación y Early Stopping
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item() * inputs.size(0)
                val_correct += calculate_accuracy(outputs, labels) * inputs.size(0)
                val_total += labels.size(0)

        val_loss /= val_total
        val_accuracy = val_correct / val_total

        # Almacenar las métricas de validación
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        lr_scheduler.step(val_loss)

        # Comprobar si este modelo es el mejor hasta ahora
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            if best_accuracy > global_best_accuracy:
                global_best_accuracy = best_accuracy
                # Guardar el nuevo valor de global_best_accuracy
                with open(best_accuracy_file, "w") as file:
                    json.dump(global_best_accuracy, file)
                torch.save(model.state_dict(), best_model_path)  # Sobreescribir el mejor modelo
                plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies, plot_file_name_base)  # Sobreescribir el gráfico
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve == early_stopping_patience:
            print("Early stopping")
            break

    print(f"Trial {trial.number} completado. Mejor precisión en validación: {best_accuracy}")

    # Limpiar la memoria después de cada trial
    del model, optimizer
    torch.cuda.empty_cache()
    gc.collect()

    # Devolver la pérdida en lugar de la precisión
    return best_accuracy


In [12]:
input_shape = features_train.shape[1]  # Número de variables explicativas

In [13]:
import optuna

storage = "sqlite:///example.db"  # Ruta de la base de datos SQLite
study_name = "accuracy_gpu_study"  # Nuevo nombre del estudio

# Crear un nuevo estudio
study = optuna.create_study(study_name=study_name, direction='maximize', storage=storage, load_if_exists=True)

/home/21953404Victor/SDC3/tutorial-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-11-30 15:15:15,617] Using an existing study with name 'accuracy_gpu_study' instead of creating a new one.


In [14]:
# Número de ensayos en el estudio
num_trials = len(study.trials)
print(f"El estudio contiene {num_trials} ensayos.")

El estudio contiene 478 ensayos.


In [ ]:
trials_restantes=50000
study.optimize(objective, n_trials=trials_restantes)

Usando GPU para el Trial 377
Entrenando Trial 377...


[I 2023-11-29 18:35:43,545] Trial 377 finished with value: 0.8799387442572741 and parameters: {'n_layers': 7, 'n_units_l0': 1024, 'activation_l0': 'LeakyReLU', 'dropout_l0': 0.20586549895108144, 'batch_norm_l0': False, 'l2_reg_l0': 0.000542687177346643, 'leakyrelu_alpha_l0': 0.17182393242401497, 'n_units_l1': 128, 'activation_l1': 'LeakyReLU', 'dropout_l1': 0.12076272856367021, 'batch_norm_l1': False, 'l2_reg_l1': 0.007205391117408795, 'leakyrelu_alpha_l1': 0.045352126597739906, 'n_units_l2': 1024, 'activation_l2': 'ReLU', 'dropout_l2': 0.4995547709991217, 'batch_norm_l2': False, 'l2_reg_l2': 2.8634331682601625e-05, 'n_units_l3': 1024, 'activation_l3': 'LeakyReLU', 'dropout_l3': 0.11157486962628477, 'batch_norm_l3': True, 'l2_reg_l3': 0.0037526333590269796, 'leakyrelu_alpha_l3': 0.1686995710767366, 'n_units_l4': 64, 'activation_l4': 'LeakyReLU', 'dropout_l4': 0.29339315201188676, 'batch_norm_l4': False, 'l2_reg_l4': 0.00038962626267814786, 'leakyrelu_alpha_l4': 0.24224093944067351, 'n_

Early stopping
Trial 377 completado. Mejor precisión en validación: 0.8799387442572741
Usando GPU para el Trial 378
Entrenando Trial 378...


Epoch 00038: reducing learning rate of group 0 to 1.2187e-04.


Epoch 00071: reducing learning rate of group 0 to 4.2581e-05.


...

Early stopping
Trial 475 completado. Mejor precisión en validación: 0.8477794793261868
Usando GPU para el Trial 476
Entrenando Trial 476...


Early stopping
Trial 476 completado. Mejor precisión en validación: 0.8508422664624808


[I 2023-11-29 20:52:29,493] Trial 476 finished with value: 0.8508422664624808 and parameters: {'n_layers': 5, 'n_units_l0': 2048, 'activation_l0': 'LeakyReLU', 'dropout_l0': 0.01838169322244565, 'batch_norm_l0': False, 'l2_reg_l0': 0.003194450893772112, 'leakyrelu_alpha_l0': 0.29591512577143225, 'n_units_l1': 512, 'activation_l1': 'Tanh', 'dropout_l1': 0.39133266799429467, 'batch_norm_l1': False, 'l2_reg_l1': 0.0013094439881951403, 'n_units_l2': 256, 'activation_l2': 'SELU', 'dropout_l2': 0.32501685753310305, 'batch_norm_l2': True, 'l2_reg_l2': 0.000502030031715487, 'n_units_l3': 2048, 'activation_l3': 'LeakyReLU', 'dropout_l3': 0.2863027512309723, 'batch_norm_l3': True, 'l2_reg_l3': 0.0028084785745685938, 'leakyrelu_alpha_l3': 0.15114888483118913, 'n_units_l4': 64, 'activation_l4': 'LeakyReLU', 'dropout_l4': 0.39728217760909046, 'batch_norm_l4': True, 'l2_reg_l4': 2.1171973939627343e-05, 'leakyrelu_alpha_l4': 0.24607608998628866, 'optimizer': 'RMSprop', 'lr': 0.0035417509101349823, 'r

Usando GPU para el Trial 477
Entrenando Trial 477...


Epoch 67/414:  17%|███████████████████████████████▉                                                                                                                                                                 | 304/1837 [00:00<00:04, 373.53it/s]

In [15]:
print('Número de pruebas finalizadas:', len(study.trials))
print('Mejor prueba:', study.best_trial.params)

Número de pruebas finalizadas: 478
Mejor prueba: {'activation_l0': 'LeakyReLU', 'activation_l1': 'Tanh', 'activation_l2': 'ReLU', 'activation_l3': 'LeakyReLU', 'activation_l4': 'LeakyReLU', 'activation_l5': 'LeakyReLU', 'batch_norm_l0': False, 'batch_norm_l1': False, 'batch_norm_l2': True, 'batch_norm_l3': True, 'batch_norm_l4': True, 'batch_norm_l5': False, 'batch_size': 128, 'dropout_l0': 0.15555416562446076, 'dropout_l1': 0.04710073432465321, 'dropout_l2': 0.47138973051951627, 'dropout_l3': 0.03696926911573405, 'dropout_l4': 0.3663598253395825, 'dropout_l5': 0.0760906198358584, 'early_stopping_patience': 19, 'epochs': 1828, 'l2_reg_l0': 0.0006237200063191363, 'l2_reg_l1': 0.0041501135567624334, 'l2_reg_l2': 5.418952601060645e-05, 'l2_reg_l3': 0.0038582256308527614, 'l2_reg_l4': 0.0003165493944413712, 'l2_reg_l5': 1.1447128613987452e-05, 'leakyrelu_alpha_l0': 0.28934819519495897, 'leakyrelu_alpha_l3': 0.09538751187985593, 'leakyrelu_alpha_l4': 0.29991298638602004, 'leakyrelu_alpha_l5

## Test

In [16]:
import torch.nn as nn

class PyTorchModel(nn.Module):
    def __init__(self, params, input_shape):
        super(PyTorchModel, self).__init__()
        self.layers = nn.ModuleList()
        self.l2_regs = []
        current_shape = input_shape

        # Definir la arquitectura de la red usando los hiperparámetros fijos
        n_layers = params['n_layers']
        for i in range(n_layers):
            num_units = params[f'n_units_l{i}']
            activation = params[f'activation_l{i}']
            dropout_rate = params[f'dropout_l{i}']
            use_batch_norm = params[f'batch_norm_l{i}']
            l2_reg = params[f'l2_reg_l{i}']

            # Crear y añadir la capa lineal
            layer = nn.Linear(current_shape, num_units)
            self.layers.append(layer)
            self.l2_regs.append(l2_reg)  # Añadir la tasa de regularización L2

            current_shape = num_units

            # Añadir Batch Normalization si se indica
            if use_batch_norm:
                self.layers.append(nn.BatchNorm1d(num_units))
            
            # Añadir Dropout
            self.layers.append(nn.Dropout(dropout_rate))

            # Añadir la activación
            if activation == 'LeakyReLU':
                negative_slope = params.get(f'leakyrelu_alpha_l{i}', 0.01)  # Valor predeterminado si no se proporciona
                self.layers.append(nn.LeakyReLU(negative_slope=negative_slope))
            else:
                self.layers.append(getattr(nn, activation)())

        # Capa de salida
        self.output = nn.Linear(current_shape, 1)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return torch.sigmoid(self.output(x))

    def get_l2_loss(self):
        l2_loss = 0.0
        for layer, l2_reg in zip(self.layers, self.l2_regs):
            if isinstance(layer, nn.Linear):
                l2_loss += l2_reg * layer.weight.norm(2)
        return l2_loss


In [19]:
import torch

# Cargar el estado del modelo
state_dict = torch.load("best_model.pt")

# Imprimir la estructura del modelo
print(state_dict.keys())


odict_keys(['layers.0.weight', 'layers.0.bias', 'layers.3.weight', 'layers.3.bias', 'layers.6.weight', 'layers.6.bias', 'layers.7.weight', 'layers.7.bias', 'layers.7.running_mean', 'layers.7.running_var', 'layers.7.num_batches_tracked', 'layers.10.weight', 'layers.10.bias', 'layers.11.weight', 'layers.11.bias', 'layers.11.running_mean', 'layers.11.running_var', 'layers.11.num_batches_tracked', 'output.weight', 'output.bias'])


In [18]:
model = PyTorchModel(study.best_trial.params, input_shape)
model.load_state_dict(torch.load("best_model.pt"))
model.eval()  # Poner el modelo en modo de evaluación

RuntimeError: Error(s) in loading state_dict for PyTorchModel:
	Missing key(s) in state_dict: "layers.14.weight", "layers.14.bias", "layers.15.weight", "layers.15.bias", "layers.15.running_mean", "layers.15.running_var", "layers.18.weight", "layers.18.bias". 
	size mismatch for layers.6.weight: copying a param with shape torch.Size([2048, 512]) from checkpoint, the shape in current model is torch.Size([1024, 512]).
	size mismatch for layers.6.bias: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for layers.7.weight: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for layers.7.bias: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is torch.Size([1024]).
	size mismatch for layers.7.running_mean: copying a param with shape torch.Size([2048]) from checkpoint, the shape in current model is tor

In [ ]:
def evaluate_model(model, criterion, test_loader, device):
    model.eval()  # Poner el modelo en modo de evaluación
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # Desactivar el cálculo de gradientes
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            test_loss += criterion(outputs, labels).item()  # Sumar la pérdida del lote
            predicted = outputs.round()  # Redondear las salidas para obtener predicciones binarias
            correct += (predicted == labels).sum().item()  # Sumar predicciones correctas
            total += labels.size(0)

    test_loss /= len(test_loader)
    accuracy = correct / total
    return test_loss, accuracy

# Suponiendo que tienes un DataLoader para el conjunto de prueba llamado 'test_loader'
test_loss, test_accuracy = evaluate_model(model, criterion, test_loader, device)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

# Asegurarse de que el modelo esté en modo de evaluación y moverlo al dispositivo adecuado
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Realizar las predicciones
with torch.no_grad():
    inputs = features_test.to(device)
    outputs = model(inputs)
    predictions = outputs.round().cpu().numpy()  # Redondear las predicciones y moverlas al CPU

# Calcular y visualizar la matriz de confusión
conf_matrix = confusion_matrix(labels_test.numpy(), predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues')
plt.title('Matriz de Confusión')
plt.xlabel('Predicciones')
plt.ylabel('Valores Reales')
plt.show()

# Calcular y mostrar otras métricas
accuracy = accuracy_score(labels_test.numpy(), predictions)
recall = recall_score(labels_test.numpy(), predictions)
precision = precision_score(labels_test.numpy(), predictions)
f1 = f1_score(labels_test.numpy(), predictions)
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")